In [1]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
import copy
import random

# # 获取数据部分

In [30]:
# 原始文件路径 lmp文件
file_path = r'C:\Users\zhang\Desktop\桌面资料\论文\AI_test\26k-paper_data\26kcry_1000p.lmp'

# 是否将这个npy数据保存
Is_save = 1
# 输出npy文件的地址
save_npy_path = r'C:\Users\zhang\Desktop\桌面资料\论文\AI_test\26k-paper_data\26kcry_0p.npy'

########################### 
# 这批数据是0 glass 是 1crystal
label = 1

# 分析帧数 
ana_frame_num = 0

# 选取多少个数据出来一般是1000  如果是‘-1’ 表示全部   # -1表示原来的xyz还得给出来
generate_num = -1

# 是否需要输出原文件的xyz格式
Is_save_xyz = 1
# 输出源文件的xyz
save_xyz_path = r'C:\Users\zhang\Desktop\桌面资料\论文\AI_test\26k-paper_data\pre_2600k_0p.xyz'


# dis距离
cut_off_dis  = 5.0
# 设置中心原子
center_atom_type = 2
# 设置临近原子类型
neighber_type_list = [1,2]
# 一个数据中的原子总数
neighber_atom_num = 45

In [22]:
# 函数体
# 每一个原子的信息
class Atom:
    def __init__(self, id=-1, type=-1, q=-1, x=-1, y=-1, z=-1, vx=-1, vy=-1, vz=-1, fx=-1, fy=-1, fz=-1, c_ke_atom=-1, c_pe_atom=-1, v_e_total=-1):
        self.id = id
        self.type = type
        self.x = x
        self.y = y
        self.z = z
        self.q = q
        self.c_ke_atom = c_ke_atom
        self.c_pe_atom = c_pe_atom
        self.v_e_total = v_e_total
        self.vx = vx
        self.vy = vy
        self.vz = vz
        self.fx = fx
        self.fy = fy
        self.fz = fz
# 计算ppp中的距离
def cal_ppp_distance(pos1, pos2, box_size):
    # 计算两个位置之间的最小距离，考虑周期性边界条件
    pos1,pos2,box_size = np.array(pos1),np.array(pos2),np.array(box_size)
    # 计算位置差矢量
    delta = pos2 - pos1
    
    # 应用最小图像约束，确保距离在周期内最短
    delta = delta - np.round(delta / box_size) * box_size
    # 计算距离
    distance = np.linalg.norm(delta)
    if distance<1e-4:
        distance = float('inf')
    return distance
# 转换ppp中的坐标
def tran_position(center,pos1,box_size):
    center,pos1,box_size = np.array(center),np.array(pos1),np.array(box_size)
    delta = pos1 - center
    pos1 = pos1 - np.round(delta/box_size)*box_size
    return pos1-center
# 旋转原子
def rotate_atoms(positions, angles):
    # 将具有 XYZ 坐标的原子群分别绕 X、Y、Z 轴旋转给定角度
    
    # 将角度转换为弧度
    angles_rad = np.deg2rad(angles)
    
    # 初始化旋转矩阵为单位矩阵
    rotation_matrix = np.eye(3)
    
    # 分别计算绕 X、Y、Z 轴的旋转矩阵，并依次应用
    for i in range(3):
        axis = np.zeros(3)
        axis[i] = 1
        rotation_matrix_i = get_rotation_matrix(axis, angles_rad[i])
        rotation_matrix = np.dot(rotation_matrix_i, rotation_matrix)
    
    # 应用旋转矩阵到每个原子的坐标
    rotated_positions = np.dot(positions, rotation_matrix.T)
    
    return rotated_positions

def get_rotation_matrix(axis, angle):
    # 根据给定轴和角度获取绕轴旋转的旋转矩阵
    
    axis = axis / np.linalg.norm(axis)  # 将轴向量归一化
    
    c = np.cos(angle)
    s = np.sin(angle)
    t = 1 - c
    
    rotation_matrix = np.array([[t*axis[0]*axis[0]+c, t*axis[0]*axis[1]-s*axis[2], t*axis[0]*axis[2]+s*axis[1]],
                                [t*axis[0]*axis[1]+s*axis[2], t*axis[1]*axis[1]+c, t*axis[1]*axis[2]-s*axis[0]],
                                [t*axis[0]*axis[2]-s*axis[1], t*axis[1]*axis[2]+s*axis[0], t*axis[2]*axis[2]+c]])
    
    return rotation_matrix
def draw_flower():
    pattern = [
        " 🌹🌹🌹  🌹   🌹    🌹 ",
        " 🌹  🌹  🌹  🌹     🌹 ",
        " 🌹  🌹  🌹🌹🌹     🌹 ",
        " 🌹  🌹  🌹   🌹       ",
        " 🌹🌹🌹  🌹    🌹   🌹  "
    ]

    for row in pattern:
        print(row)

In [23]:
Train_data,Train_label = [],[]

# 所有信息储存的总列表
list_all_frames = []

# 读取lmp文件
# 打开文件并读取所有行数据
with open(file_path, 'r') as file:
    lines = file.readlines()
# 每一页的原子数
num_atom =int(lines[3])
# 定义一页的行数
lines_per_page = num_atom + 9
# 计算总页数
total_pages = (len(lines) + lines_per_page - 1) // lines_per_page
# 遍历每一页
for page in range(total_pages):
    # 计算当前页的起始行号和结束行号
    start_line = page * lines_per_page
    end_line = start_line + lines_per_page
    # 从总行数据中获取当前页的行数据
    page_lines = lines[start_line:end_line]
    ## 开始分析每一页中的数据
    # 每一页的数据存储在这个list中
    list_frame = []
    #print("Current frame number :",page)
    # 每一页的第九行开始才是原子数据
    for i_atom in page_lines[9:]:
        ii = i_atom.split()
        atom = Atom(int(ii[0]),int(ii[1]),float(ii[2]),float(ii[3]),float(ii[4]),float(ii[5]),float(ii[6]),float(ii[7]),float(ii[8])\
                    ,float(ii[9]),float(ii[10]),float(ii[11]))
#         atom = Atom(int(ii[0]),int(ii[1]),float(ii[2]),float(ii[3]),float(ii[4]),float(ii[5]),float(ii[6]),float(ii[7]),float(ii[8])\
#                     ,float(ii[9]),float(ii[10]),float(ii[11]),float(ii[12]),float(ii[13]),float(ii[14]))
        
        list_frame.append(atom)
    list_all_frames.append(list_frame)
# 输出文档中的其他信息
print('\nResult analysis completed')
print("Other information:")
print("Number of total frames:",total_pages)
print("Number of atoms:", lines[3].split()[0])
print('Box size:',round(float(lines[5].split()[1]),3))



Result analysis completed
Other information:
Number of total frames: 1001
Number of atoms: 15000
Box size: 108.942


In [24]:
# 将要分析的列表单独独立出来 深拷贝
All_atom_list = copy.deepcopy(list_all_frames[ana_frame_num])
# 有两个列表 一个是center 一个是neighber
Center_list = []
# 先得出最大的BOX距离 并且这个BOX的xyz可以不相等
x_max,y_max,z_max = 0,0,0
for i in All_atom_list:
    # 将要中心的原子种类放进列表
    if i.type == center_atom_type:
        Center_list.append(i)
    if i.x>x_max:
         x_max = i.x
    if i.y>y_max:
        y_max = i.y
    if i.z>z_max:
        z_max = i.z
Box_size = np.array([x_max,y_max,z_max])


In [25]:
# 如果是-1 就全部取出来 否则随机选择center原子
if generate_num==-1:
    random_center_list = Center_list
else:
    random_center_list = random.sample(Center_list,generate_num)
# 每次处理一个原子的数据
for center in random_center_list:
    center_atom_position = np.array([center.x, center.y,center.z])
    neighber_atom_list = []
    #############找出附近原子#################
    for i in range(len(All_atom_list)):
        # 必须要这个原子的类型在特定的表中才分析
        if All_atom_list[i].type in neighber_type_list:
            # 只有两个原子的距离小于dis才
            temp_neigh_list = [All_atom_list[i].type, All_atom_list[i].x, All_atom_list[i].y, All_atom_list[i].z]
            if cal_ppp_distance(center_atom_position, temp_neigh_list[1:], Box_size) < cut_off_dis:
                neighber_atom_list.append(temp_neigh_list)
    neighber_atom_list = np.array(neighber_atom_list)
    ####### 标准化数据 #################
    neighber_type = neighber_atom_list[:,:1].reshape(-1)-1
    # 平移坐标到原点
    tran_zero_neighber = tran_position(center_atom_position,neighber_atom_list[:,1:],Box_size)   
    # xyz三个角随机旋转0-180度
    random_angle = [random.random() * 180,random.random() * 180,random.random() * 180]   
    tran_rotate_neighber = rotate_atoms(tran_zero_neighber, random_angle)    
    # 再加上标签 凑齐数据量 xyz和type
    final_data = np.zeros(shape=(neighber_atom_num,4))
    # 如果数据量比标准值大 后边的就不要了
    for i in range(min(len(neighber_atom_list),neighber_atom_num)):
        final_data[i] = [tran_rotate_neighber[i][0],tran_rotate_neighber[i][1],tran_rotate_neighber[i][2],neighber_type[i]]
    #############再写入文件##################
    # 如果是训练集 就加上自定义的标签
    Train_data.append(final_data)
    Train_label.append(label)
    
print('数据准备完成！')

数据准备完成！


In [28]:
data = []
# 训练集写入

data = Train_data
np.save(save_npy_path,data)
print('npy文件已经保存在文件夹中！')

npy文件已经保存在文件夹中！


In [31]:
# 表示这是要预测的数据，得输出其原本的xyz文件
if generate_num == -1 and Is_save_xyz == 1:
    # 写回文件
    with open(save_xyz_path,'w') as file:
        file.write('15000\n')
        file.write('Atoms\n')
        # 写回原子
        for i in All_atom_list:
            if i.type == 1: 
                    file.write('O    '+str(i.x)+'    '+str(i.y)+'    '+str(i.z)+'\n')
            elif i.type == 2:
                    file.write('Nb    '+str(i.x)+'    '+str(i.y)+'    '+str(i.z)+'\n')
            elif i.type == 3:
                    file.write('Li    '+str(i.x)+'    '+str(i.y)+'     '+str(i.z)+'\n')
        
    print('xyz文件写完')
draw_flower()

xyz文件写完
 🌹🌹🌹  🌹   🌹    🌹 
 🌹  🌹  🌹  🌹     🌹 
 🌹  🌹  🌹🌹🌹     🌹 
 🌹  🌹  🌹   🌹       
 🌹🌹🌹  🌹    🌹   🌹  
